Parallelizing Neural Network
Training with TensorFlow

####  This chapter begins the next stage of our journey into training machine learning and
#### deep learning, and we'll explore the following topics:

    . How TensorFlow improves training performance
    . Working with TensorFlow to write optimized machine learning code
    . Using TensorFlow high-level APIs to build a multilayer neural network
    . Choosing activation functions for artificial neural networks
    . Introducing Keras, a high-level wrapper around TensorFlow, for
        implementing common deep learning architectures most conveniently

#### pip install tensorflow
 

In [7]:
import tensorflow as tf


In [8]:
# # Create a grap
# g = tf.Graph()
# with g.as_default():
#     x = tf.placeholder(dtype=tf.float32,
#                        shape=(None), name='x')
#     w = tf.variable(2.0, name='weight')
#     b = tf.variable(0.7, name='bias')
#     z = w*x + b

#     init = tf.global_variables_initializer()

# # Create a session and pass in graph g
# with tf.session(graph=g) as sess:
#     # Initialize all variables
#     sess.run(init)
#     # Print z given some input x
#     for t in [1.0, 0.6, -1.8]:
#         print('x=%4.1f --> z=%4.1f'%(t, sess.run(z, feed_dict={x:t})))



import tensorflow as tf

# Create variables
w = tf.Variable(2.0, name='weight')
b = tf.Variable(0.7, name='bias')

# Define a function for the computation
@tf.function
def compute_z(x):
    z = w * x + b
    return z

# Initialize variables
w.assign(2.0)
b.assign(0.7)

# Run the computation
for t in [1.0, 0.6, -1.8]:
    z = compute_z(t)
    print('x=%4.1f --> z=%4.1f' % (t, z))


x= 1.0 --> z= 2.7
x= 0.6 --> z= 1.9
x=-1.8 --> z=-2.9


 Developing simple model with the low-level Tensorflow api

In [1]:
import tensorflow as tf
import numpy  as np

X_train = np.arange(10).reshape((10, 1))
y_train = np.array([1.0, 1.3, 3.1,2.0, 5.0, 6.3, 6.6, 7.4, 8.0, 9.0])

print(X_train , y_train)

[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]] [1.  1.3 3.1 2.  5.  6.3 6.6 7.4 8.  9. ]


In [16]:
class TfLinreg(object):
    def __init__(self, x_dim, learning_rate=0.01, random_seed=None):
        self.x_dim = x_dim
        self.learning_rate = learning_rate
        self.g = tf.Graph()

        # Build the model
        with self.g.as_default():
            ## set graph-level random_seed
            tf.random.set_seed(random_seed)

    def build(self):
            ## Create initializer
            self.init_op = tf.global_variables_initializer()

    def build(self):
        ## define placeholders for  input 
        self.X = tf.keras.Input(shape=(None, self.x_dim), name='x_input')
        self.y = tf.keras.Input(shape=(None,), name='y_input')

        print(self.X)
        print(self.y)
        ## define weight matrix and bias vector
        w = tf.Variable(tf.zeros(shape=(1)), name='weight')
        b = tf.Variable(tf.zeros(shape=(1)), name='bias')

        print(w)
        print(b)

        ## define model operations
        self.z_net = tf.squeeze(w * self.X + b, name='z_net')
        print(self.z_net)

        sqr_errors = tf.square(self.y - self.z_net,name='sqr_errors')
        print(sqr_errors)
        
        self.mean_cost = tf.reduce_mean(sqr_errors,name='mean_cost')
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate,name='GradientDescent')
        
        self.optimizer = optimizer.minimize(self.mean_cost)
lrmodel = TfLinreg(x_dim=X_train.shape[1], learning_rate=0.01)


In [19]:

def train_linreg(model, X_train, y_train, num_epochs=10):
    # Initialize all variables: W and b

    # List to store the training costs
    training_costs = []
    
    # Training loop for the specified number of epochs
    for epoch in range(num_epochs):
        # Use a GradientTape for automatic differentiation
        with tf.GradientTape() as tape:
            predictions = model(X_train, training=True)
            cost = model.mean_cost(y_train, predictions)
        
        # Compute gradients and apply them
        gradients = tape.gradient(cost, model.trainable_variables)
        model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        # Append the cost to the training_costs list
        training_costs.append(cost.numpy())
    
    # Return the list of training costs
    return training_costs

In [20]:

train_linreg(lrmodel, X_train, y_train)

TypeError: 'TfLinreg' object is not callable

Building multilayer neural networks using
TensorFlow's Layers API

In [25]:
import os
import struct

def load_mnist(path, kind='train'):
    "Load MNIST data from 'path' "
    labels_path = os.path.join(path, '%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images-idx3-ubyte' % kind)

    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
        images = ((images / 255.) -.5) * 2

    return images, labels


In [27]:
import os
import struct
import numpy as np

def load_mnist(path, kind='train'):
    """
    Load MNIST data from `path`.

    Parameters:
    path (str): The directory containing the MNIST data files.
    kind (str): Either 'train' or 't10k' to specify training or test data.

    Returns:
    tuple: A tuple containing:
        - images (np.ndarray): The image data as a 2D numpy array of shape (num_samples, 784).
        - labels (np.ndarray): The labels as a 1D numpy array of shape (num_samples,).
    """
    # Paths to the labels and images files
    labels_path = os.path.join(path, f'{kind}-labels-idx1-ubyte')
    images_path = os.path.join(path, f'{kind}-images-idx3-ubyte')

    # Load labels
    try:
        with open(labels_path, 'rb') as lbpath:
            magic, n = struct.unpack('>II', lbpath.read(8))
            labels = np.fromfile(lbpath, dtype=np.uint8)
    except FileNotFoundError as e:
        raise FileNotFoundError(f"Labels file not found at {labels_path}") from e
    except Exception as e:
        raise RuntimeError(f"Error reading labels file: {e}") from e

    # Load images
    try:
        with open(images_path, 'rb') as imgpath:
            magic, num, rows, cols = struct.unpack(">IIII", imgpath.read(16))
            images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), rows * cols)
            images = ((images / 255.) - 0.5) * 2  # Normalize to [-1, 1]
    except FileNotFoundError as e:
        raise FileNotFoundError(f"Images file not found at {images_path}") from e
    except Exception as e:
        raise RuntimeError(f"Error reading images file: {e}") from e

    return images, labels

# Example usage:
# path = 'path/to/mnist/dataset'
# images, labels = load_mnist(path, kind='train')


In [28]:
X_train, y_train = load_mnist('./mnist/', kind='train')
print('Rows: %d, Columns: %d' %(X_train.shape[0],X_train.shape[1]))

RuntimeError: Error reading labels file: [Errno 13] Permission denied: './mnist/train-labels-idx1-ubyte'